# Classic Binomial model

Binomial model describes 
underlying with discrete time 
Markov process. 

This Markov chain is fully parametrized with: 
- $S_0 \in \mathbb{R}$: starting price 
- $u>1$: upwards multiplicative factor 
- $d<1$: downwards multiplicative factor 
- $q>1$: average increase factor 

in the following way:
<!-- - State space $\{u^{i} d^{j} S_0 | i,j \in \mathbb{N}\}$ -->
- Starting in state $S_0$ 
- Transition rule  $\forall n,i,j \in \mathbb{N}:$
$$
\begin{aligned}
P(S_{n+1} = u^{i+1} d^{j} S_0 | S_{n} = u^{i} d^{j} S_0) &= p \\
P(S_{n+1} =u^{i} d^{j+1} S_0 | S_{n} = u^{i} d^{j} S_0) &= 1-p
.
\end{aligned}
$$ 
i.e p is the probability of going upwards and can be derived:
$$
\begin{aligned}
E[S_{n+1} | S_{n}] &= q S_n \Leftrightarrow \\
p&= \frac{q-d}{u-d}
.
\end{aligned}
$$ 
With some elementary probability theory we find
$ \forall n,i \in \mathbb{N}$: 

$$
P(S_{n} =u^{i} d^{n-i} S_0) = 
\left( \begin{matrix} {{{n}}} \\ {{{i}}} \\ \end{matrix} \right) p^{i} \left( 1-p \right)^{n-i}. 
$$ 
i.e going upwards is Binomial distributed or $S_{n} = u^{X} d^{n-X} S_0$ with $X \sim \text{Bin}(n,p)$.


# Comparing to BS assumptions
We will not be doing any calibration to empirical data. Calibration 
defined as choosing model parameters such that a model behaves  
as similar as possible to the empirical data. <br>
But we will compare the Binomial model to
GBM. GBM is given by following 
SDE: 
$$
\frac{dS_{t}}{S_{t}} = r dt + \sigma dW_{t}.
$$ 
with initial condition $S_0 \in \mathbb{R}^{+}$, $r>0$ (BS)-intrest rate and $\sigma>0$ BS-volatility.
Which has known solution:
$$
S_{t}=S_{0} \operatorname{e x p} \biggl( \left( r-{\frac{\sigma^{2}} {2}} \right) t+\sigma W_{t} \biggr). 
$$
By assuming that every transition in the binomial model takes $\Delta t$ time, typical way of matching 
GBM with the binomial parameters is following way:

$$
\begin{aligned}
u=&e^{\sigma\sqrt{\Delta t}} \\
d=&e^{-\sigma\sqrt{\Delta t}} \\
q=&e^{r \Delta t}
.
\end{aligned}
$$ 

